# Taller 05: Herramientas para Indexación

**Objetivo:**

Este ejercicio te permitirá recordar los conceptos fundamentales de indexación clásica en sistemas de recuperación de información. Implementarás un índice invertido manualmente y luego explorarás herramientas como *Whoosh* y *Elasticsearch* para construir y consular índices.

### Parte 1: Construcción Manual de un Índice Invertido

**1. Cargar los datos en Python:**

Usa pandas para cargar y explorar el dataset.


In [1]:
#Bibliotecas
import pandas as pd
import re
from collections import defaultdict

In [2]:
# Cargar dataset
data = pd.read_csv("../data/wiki_movie_plots_deduped.csv")
data = data[['Title', 'Plot']].dropna()  # Asegúrate de trabajar con campos no nulos
print(data.head())

                              Title  \
0            Kansas Saloon Smashers   
1     Love by the Light of the Moon   
2           The Martyred Presidents   
3  Terrible Teddy, the Grizzly King   
4            Jack and the Beanstalk   

                                                Plot  
0  A bartender is working at a saloon, serving dr...  
1  The moon, painted with a smiling face hangs ov...  
2  The film, just over a minute long, is composed...  
3  Lasting just 61 seconds and consisting of two ...  
4  The earliest known adaptation of the classic f...  


**2. Construir un índice invertido:**

Realiza una normalización básica del texto y posteriormente se genera el índice invertido.

In [3]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar puntuación
    return text

data['Normalized_Plot'] = data['Plot'].apply(normalize_text)

In [4]:
inverted_index = defaultdict(list)

for idx, row in data.iterrows():
    tokens = row['Normalized_Plot'].split()
    for token in set(tokens):  # Evitar duplicados en un mismo documento
        inverted_index[token].append(row['Title'])

print(dict(list(inverted_index.items())[:10]))  # Muestra las primeras 10 entradas

{'nations': ['Kansas Saloon Smashers', 'Redskin', 'Gabriel Over the White House', 'Life Begins in College', "Idiot's Delight", 'The Son of Monte Cristo', 'To Be or Not to Be', 'Watch on the Rhine', 'Ministry of Fear', 'Cornered', 'Monsieur Beaucaire', 'The Stranger', 'The Search', 'The Red Danube', 'Reign of Terror', 'Peking Express', 'When Worlds Collide', 'One Minute to Zero', 'The 49th Man', 'The Glass Wall', 'Dragonfly Squadron', 'Gunfighters of the Northwest', 'Target Zero', 'Godzilla, King of the Monsters!', 'The 27th Day', 'A Face in the Crowd', 'The Giant Claw', 'The Colossus of New York', 'Donald in Mathmagic Land', 'Invisible Invaders', 'North by Northwest', 'The World, the Flesh and the Devil', 'Atlantis, the Lost Continent', 'Bridge to the Sun', 'Romanoff and Juliet', 'The Manchurian Candidate', 'Panic in Year Zero!', 'The Road to Hong Kong', 'Carol for Another Christmas', 'Circus World', 'The Fall of the Roman Empire', 'First Men in the Moon', 'A Global Affair', 'Cast a Gi

**3. Realizar consultas en el índice:**

Implementa una función para buscar palabras clave.

In [5]:
def search_inverted_index(query, index):
    query = normalize_text(query)
    return index.get(query, [])

print(search_inverted_index("cyborg", inverted_index))

['The Colossus of New York', 'Cyborg 2087', 'Spacehunter: Adventures in the Forbidden Zone', 'Superman III', 'Warrior of the Lost World', 'The Terminator', 'RoboCop', 'Cyborg', 'Moontrap', 'Nemesis', 'Cyborg 3: The Recycler', 'Cyborg Cop II', 'Space Truckers', 'Future War', 'Leprechaun 4: In Space', 'Virus', 'Jason X', 'Treasure Planet', 'Godzilla: Final Wars', 'Star Wars: Episode III – Revenge of the Sith', 'Tekken', 'Terminator Salvation', 'Justice League: The Flashpoint Paradox', 'Superman: Unbound', 'RoboCop', 'Hardcore Henry', 'Logan', 'Fortress', 'The Machine', 'Kill Command', 'April and the Extraordinary World', 'Sixty Million Dollar Man', 'Kung Fu Cyborg', 'Future X-Cops', 'Godzilla vs. Megalon', 'Kamen Rider V3', 'Kamen Rider V3 vs. the Destron Monsters', 'Terror of Mechagodzilla', 'JAKQ Dengeki Tai', 'JAKQ Dengeki Tai vs. Goranger', 'Sun Vulcan Movie', '964 Pinocchio', 'Zeiram', 'Tetsuo II: Body Hammer', 'Ghost in the Shell', 'Godzilla Against Mechagodzilla', 'Godzilla: Tokyo

### Parte 2: Usar Whoosh para Indexación y Recuperación

In [9]:
!pip install whoosh

   ---------------------------------------- 0.0/468.8 kB ? eta -:--:--
   -- ------------------------------------ 30.7/468.8 kB 660.6 kB/s eta 0:00:01
   ----- --------------------------------- 61.4/468.8 kB 656.4 kB/s eta 0:00:01
   --------- ---------------------------- 122.9/468.8 kB 901.1 kB/s eta 0:00:01
   --------------------- ------------------ 256.0/468.8 kB 1.3 MB/s eta 0:00:01
   ------------------------------ --------- 358.4/468.8 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 468.8/468.8 kB 1.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**1. Configuración del índice con Whoosh:**

Define un esquema y configura el índice.

In [6]:
from whoosh.fields import Schema, TEXT
from whoosh.index import create_in
import os

schema = Schema(Title=TEXT(stored=True), Plot=TEXT(stored=True))

if not os.path.exists("index"):
    os.mkdir("index")
ix = create_in("index", schema)

**2. Agregar documentos al índice:**

Agrega los títulos y tramas.

In [7]:
from whoosh.writing import AsyncWriter

writer = AsyncWriter(ix)
for _, row in data.iterrows():
    writer.add_document(Title=row['Title'], Plot=row['Plot'])
writer.commit()

**3. Realizar consultas:**

Consulta palabras clave.

In [8]:
from whoosh.qparser import QueryParser

with ix.searcher() as searcher:
    query = QueryParser("Plot", ix.schema).parse("cyborg")
    results = searcher.search(query)
    for result in results:
        print(result['Title'])

Future War
Space Truckers
JAKQ Dengeki Tai
JAKQ Dengeki Tai vs. Goranger
Kung Fu Cyborg
Future X-Cops
Kamen Rider V3
Kamen Rider V3 vs. the Destron Monsters
Cyborg She
Cyborg She


### Parte 3: Usar Elasticsearch para Indexación y Recuperación

**1. Inicia Elasticsearch con Docker:**

Configura e inicia un contenedor.

In [2]:
!pip install elasticsearch

   ---------------------------------------- 0.0/571.2 kB ? eta -:--:--
   -- ------------------------------------- 30.7/571.2 kB 1.4 MB/s eta 0:00:01
   ------- -------------------------------- 112.6/571.2 kB 1.1 MB/s eta 0:00:01
   --------------- ------------------------ 225.3/571.2 kB 1.4 MB/s eta 0:00:01
   ----------------------- ---------------- 337.9/571.2 kB 1.6 MB/s eta 0:00:01
   ------------------------------------- -- 532.5/571.2 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 571.2/571.2 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.5 kB ? eta -:--:--
   ---------------------------------------- 64.5/64.5 kB ? eta 0:00:00
   ---------------------------------------- 0.0/126.3 kB ? eta -:--:--
   ---------------------------------------- 126.3/126.3 kB 7.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/164.9 kB ? eta -:--:--
   ---------------------------------------- 164.9/164.9 kB 9.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**2. Configurar el cliente en Python:**

Usa la librería elasticsearch.


In [9]:
from elasticsearch import Elasticsearch

# Conectar a Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Verifica la conexión
if es.ping():
    print("Conexión exitosa a Elasticsearch")
else:
    print("Error al conectar con Elasticsearch")

Error al conectar con Elasticsearch


**3. Indexar documentos:**


Inserta películas en el índice.

In [ ]:
index_name = "movies"

# Define el esquema del índice
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body={
        "mappings": {
            "properties": {
                "Title": {"type": "text"},
                "Plot": {"type": "text"}
            }
        }
    })
    print(f"Índice '{index_name}' creado.")
else:
    print(f"Índice '{index_name}' ya existe.")

In [ ]:
from tqdm import tqdm

# Indexar cada película
for _, row in tqdm(data.iterrows(), total=data.shape[0]):
    doc = {
        "Title": row["Title"],
        "Plot": row["Plot"]
    }
    es.index(index=index_name, body=doc)
print("Documentos indexados.")

**4.Realizar consultas:**

Consulta con palabras clave.

In [ ]:
query = {
    "query": {
        "match": {
            "Plot": "cyborg"
        }
    }
}

results = es.search(index=index_name, body=query)
for hit in results["hits"]["hits"]:
    print(f"Título: {hit['_source']['Title']}")